<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/snowball/GS_Orden_del_D%C3%ADa_v0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import json

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d leonardocaravaggio/base-diputados

Mounted at /content/drive
100% 696M/696M [00:33<00:00, 25.1MB/s]
100% 696M/696M [00:33<00:00, 21.8MB/s]


In [6]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [7]:
base=pd.read_csv('/content/base_formateada_snowball.csv')

<ipython-input-7-83e471786f4d>:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  base=pd.read_csv('/content/base_formateada_snowball.csv')


In [9]:
import pickle

with open('/content/vocabulary_snowball.voc', 'rb') as f:
    inter = pickle.load(f)

In [10]:
base.orden=base.orden.fillna(' ')

In [11]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(2,2), vocabulary=inter)
bow_ordenes = vect.fit_transform(base['orden'])
lda = LatentDirichletAllocation(n_components=15, max_iter=50, learning_method='batch', learning_offset=50.,random_state=0)
lda.fit(bow_ordenes)

LatentDirichletAllocation(learning_offset=50.0, max_iter=50, n_components=15,
                          random_state=0)

# Por weights

In [12]:
matrix=vect.fit_transform(base['orden'])

In [13]:
disc_weights=matrix*lda.components_.transpose()
base['Topic_weight']=disc_weights.argmax(axis=1)+1

In [14]:
# Por LEGISLATURA por WEIGHTS de ORDENES DEL DIA
legislaturas=['I', 'II', 'III', 'IV', 'V', 'VI', 'VII','VIII', 'IX', 'X', 'XI', 'XII','XIII']
topics_por_legislatura=pd.DataFrame()
for x in legislaturas:
  base_legislatura=base[base.legislatura==x]
  topics_por_legislatura[x]=base_legislatura['Topic_weight'].value_counts().sort_index()
topics_por_legislatura.to_csv('topics_por_legislatura_por_weights_por_ordendeldia_snowball.csv')

In [15]:
base.fecha=pd.to_datetime(base.fecha, infer_datetime_format=True)

In [16]:
# Por AÑO por weights
topics_por_año=pd.DataFrame()
for x in range (1979, 2020):
  base_year=base[pd.DatetimeIndex(base.fecha).year==x]
  topics_por_año[x]=base_year['Topic_weight'].value_counts().sort_index()
topics_por_año.to_csv('topics_por_año_por_weights_por_ordendeldia_snowball.csv')

# Por FREX

In [17]:
import scipy as sp
from scipy.special import logsumexp
import numpy as np

def frex(mod, w=0.7):
    log_beta = np.log(mod.components_)
    log_exclusivity = log_beta - logsumexp(log_beta, axis=0)
    exclusivity_ecdf = np.apply_along_axis(ecdf, 1, log_exclusivity)
    freq_ecdf = np.apply_along_axis(ecdf, 1, log_beta)
    out = 1. / (w / exclusivity_ecdf + (1 - w) / freq_ecdf)
    return out

def ecdf(arr):
    return sp.stats.rankdata(arr, method='max') / arr.size


def t_FREX(row):
  return frex_df[matrix[row['id']].argmax()].argsort()[9]

In [18]:
base['id']=range(0,334421)
frex_df=pd.DataFrame(frex(lda))

# Se crea una columna con el topic que se asigna por frex a cada discurso de la base
base['Topic_FREX']=base.apply (lambda row: t_FREX(row), axis=1)+1

In [19]:
# Por LEGISLATURA por FREX
legislaturas=['I', 'II', 'III', 'IV', 'V', 'VI', 'VII','VIII', 'IX', 'X', 'XI', 'XII','XIII']
topics_por_legislatura=pd.DataFrame()
for x in legislaturas:
  base_legislatura=base[base.legislatura==x]
  topics_por_legislatura[x]=base_legislatura['Topic_FREX'].value_counts().sort_index()
topics_por_legislatura.to_csv('topics_por_legislatura_por_frex_por_ordendeldia_snowball.csv')

In [20]:
# Por AÑO por FREX
topics_por_año=pd.DataFrame()
for x in range (1979, 2020):
  base_year=base[pd.DatetimeIndex(base.fecha).year==x]
  topics_por_año[x]=base_year['Topic_FREX'].value_counts().sort_index()
topics_por_año.to_csv('topics_por_año_por_frex_por_ordendeldia_snowball.csv')

# A mano

In [ ]:
base['reglas y procedemientos']=0
base['Administración']=0
base['Economía']=0
base['Energía']=0
base['Seguridad']=0
base['Política Laboral']=0
base['Transporte']=0
base['Comunicaciones']=0
base['Social']=0
base['Educación']=0
base['Seguridad Social']=0
base['Justicia']=0
base['Internacionales']=0
base['Política Interna']=0
base['Otros']=0

In [ ]:
base.shape

(334421, 39)

In [ ]:
lista=[]
for row in base.values:

  # Reglas y Procedimientos
    if 'dictamen' in row[1]: row[24]=1
    if 'designacion' in row[1]: row[24]=1
    if 'decreto' in row[1]: row[24]=1
    if 'ley' in row[1]: row[24]=1
    if 'constitucion' in row[1]: row[24]=1
    if 'decisiones' in row[1]: row[24]=1
    if 'orden' in row[1]: row[24]=1
    if 'votacion' in row[1]: row[24]=1
    if 'intervencion' in row[1]: row[24]=1
    if 'nuevo gobierno' in row[1]: row[24]=1
    if 'resolucion' in row[1]: row[24]=1
    if 'modificacion' in row[1]: row[24]=1
    if 'comision' in row[1]: row[24]=1
    if 'comisión' in row[1]: row[24]=1
    if 'comi sion' in row[1]: row[24]=1
    if 'parlamentario' in row[1]: row[24]=1
    if 'presidencia' in row[1]: row[24]=1
    if 'presupuestos generales' in row[1]: row[24]=1
    if 'comunicacion del gobierno' in row[1]: row[24]=1
    if 'referendum' in row[1]: row[24]=1
    if 'eleccion' in row[1]: row[24]=1
    if 'miembros de la mesa' in row[1]: row[24]=1
    if 'diputa' in row[1]: row[24]=1
    if 'institucional' in row[1]: row[24]=1
    if 'legislativ' in row[1]: row[24]=1
    if 'mocion de censura' in row[1]: row[24]=1
    if 'constitución' in row[1]: row[24]=1
    if 'juramento' in row[1]: row[24]=1
    if 'extraordinaria' in row[1]: row[24]=1
    if 'extraorinaria' in row[1]: row[24]=1
    if 'votación de conjunto' in row[1]: row[24]=1
    if 'reglamento del congreso' in row[1]: row[24]=1
    if 'prórroga de plazo' in row[1]: row[24]=1
    if 'autoprotección de la cámara' in row[1]: row[24]=1
    if 'avocación por el pleno' in row[1]: row[24]=1
    if 'minuto de silencio' in row[1]: row[24]=1
    if 'prórrogas de plazo' in row[1]: row[24]=1
    if 'exclusión de los puntos' in row[1]: row[24]=1
    if 'canje de notas' in row[1]: row[24]=1
    if 'tramitación directa y en lectura única' in row[1]: row[24]=1
    if 'votación final' in row[1]: row[24]=1
    if 'expedi' in row[1]: row[24]=1
    if 'artícul' in row[1]: row[24]=1
    if 'constitucion interina de la cama' in row[1]: row[24]=1
    if 'designacion de la' in row[1]: row[24]=1
    if 'asuntos tratados y decisiones  adoptadas por l' in row[1]: row[24]=1
    if 'cuestiones de orden' in row[1]: row[24]=1
    if 'cumplimiento de las previsiones contenidas en' in row[1]: row[24]=1

  # Administración
    if 'gobierno' in row[1]: row[25]=1
    if 'administra' in row[1]: row[25]=1
    if 'funcionarios' in row[1]: row[25]=1
    if 'acuerdo' in row[1]: row[25]=1
    if 'ejecutivo' in row[1]: row[25]=1
    if 'publica' in row[1]: row[25]=1
    if 'minist' in row[1]: row[25]=1
    if 'el rey' in row[1]: row[25]=1
    if 'president' in row[1]: row[25]=1
    if 'secretari' in row[1]: row[25]=1

  # Economía
    if 'economi' in row[1]: row[26]=1
    if 'empresa' in row[1]: row[26]=1
    if 'desequilibrios territoriales' in row[1]: row[26]=1
    if 'presupuesto' in row[1]: row[26]=1
    if 'industria' in row[1]: row[26]=1
    if 'agricultura' in row[1]: row[26]=1
    if 'import' in row[1]: row[26]=1
    if 'export' in row[1]: row[26]=1
    if 'miner' in row[1]: row[26]=1
    if 'olivare' in row[1]: row[26]=1
    if 'pesca' in row[1]: row[26]=1
    if 'garbanzo' in row[1]: row[26]=1
    if 'sector' in row[1]: row[26]=1
    if 'precio' in row[1]: row[26]=1
    if 'siderurg' in row[1]: row[26]=1
    if 'financiac' in row[1]: row[26]=1
    if 'agraria' in row[1]: row[26]=1
    if 'impuesto' in row[1]: row[26]=1
    if 'salario' in row[1]: row[26]=1
    if 'zona franca' in row[1]: row[26]=1
    if 'plan informatico' in row[1]: row[26]=1
    if 'arrendamiento' in row[1]: row[26]=1
    if 'financier' in row[1]: row[26]=1
    if 'pesquer' in row[1]: row[26]=1
    if 'deuda' in row[1]: row[26]=1
    if 'comercio' in row[1]: row[26]=1
    if 'tribunal de cuentas' in row[1]: row[26]=1
    if 'forppa' in row[1]: row[26]=1
    if 'sociedades de garantia reciproca' in row[1]: row[26]=1
    if 'feoga' in row[1]: row[26]=1
    if 'competencia' in row[1]: row[26]=1
    if 'caucho' in row[1]: row[26]=1
    if 'fondo monetario' in row[1]: row[26]=1
    if 'inversion' in row[1]: row[26]=1
    if 'bankia' in row[1]: row[26]=1
    if 'icac' in row[1]: row[26]=1
    if 'denominaciones de origen' in row[1]: row[26]=1
    if 'montes vecinales' in row[1]: row[26]=1

  #Energía
    if 'politica energetica' in row[1]: row[27]=1
    if 'energetic' in row[1]: row[27]=1
    if 'central nuclear' in row[1]: row[27]=1
    if 'petrol' in row[1]: row[27]=1
    if 'hunosa' in row[1]: row[27]=1
    if 'embalse de jánovas' in row[1]: row[27]=1
    if 'energia' in row[1]: row[27]=1

  # Seguridad
    if 'seguridad' in row[1]: row[28]=1
    if 'proteccion' in row[1]: row[28]=1
    if 'ejercito' in row[1]: row[28]=1
    if 'policia' in row[1]: row[28]=1
    if 'carceles' in row[1]: row[28]=1
    if 'avion' in row[1]: row[28]=1
    if 'servicio civil' in row[1]: row[28]=1
    if 'arma' in row[1]: row[28]=1
    if 'atentado' in row[1]: row[28]=1
    if 'militar' in row[1]: row[28]=1
    if 'incidentes' in row[1]: row[28]=1
    if 'terroris' in row[1]: row[28]=1
    if 'misil' in row[1]: row[28]=1
    if 'tortura' in row[1]: row[28]=1
    if 'fuerzas españolas' in row[1]: row[28]=1
    if 'numero de llamada de emergencia' in row[1]: row[28]=1


  # Política laboral
    if 'politica laboral' in row[1]: row[29]=1
    if 'laboral' in row[1]: row[29]=1
    if 'patrimonio sindical' in row[1]: row[29]=1
    if 'trabaj' in row[1]: row[29]=1
    if 'empleo' in row[1]: row[29]=1
    if 'oit' in row[1]: row[29]=1

  # Transporte
    if 'aereo' in row[1]: row[30]=1
    if 'aviacion' in row[1]: row[30]=1
    if 'aeronaval' in row[1]: row[30]=1
    if 'ferrocarril' in row[1]: row[30]=1
    if 'carretera' in row[1]: row[30]=1
    if 'flota mercante' in row[1]: row[30]=1
    if 'grupo iberia' in row[1]: row[30]=1
    if 'arrastrero' in row[1]: row[30]=1
    if 'seitt' in row[1]: row[30]=1

  # Comunicaciones
    if 'televisi' in row[1]: row[31]=1
    if 'periodico' in row[1]: row[31]=1
    if 'comunicacion' in row[1]: row[31]=1
    if 'cinematog' in row[1]: row[31]=1
    if 'no-do' in row[1]: row[31]=1
    if 'tve' in row[1]: row[31]=1
    if 'telefon' in row[1]: row[31]=1

  # Sociedad
    if 'divorcio' in row[1]: row[32]=1
    if 'futbol' in row[1]: row[32]=1
    if 'dia de la' in row[1]: row[32]=1
    if 'censo' in row[1]: row[32]=1
    if 'mujer' in row[1]: row[32]=1
    if 'jornada maxima' in row[1]: row[32]=1
    if 'codigo civil' in row[1]: row[32]=1
    if 'parque nacional' in row[1]: row[32]=1

  # Educación, Ciencia y Tecnología
    if 'colegio' in row[1]: row[33]=1
    if 'enseñanza' in row[1]: row[33]=1
    if 'escolar' in row[1]: row[33]=1
    if 'educacion' in row[1]: row[33]=1
    if 'educación' in row[1]: row[33]=1
    if 'alumno' in row[1]: row[33]=1
    if 'bachillerato' in row[1]: row[33]=1
    if 'escuela' in row[1]: row[33]=1
    if 'cientifica' in row[1]: row[33]=1
    if 'educativo' in row[1]: row[33]=1
    if 'technological' in row[1]: row[33]=1


  # Seguridad Social y Ambiental
    if 'seguridad social' in row[1]: row[34]=1
    if 'inundacion' in row[1]: row[34]=1
    if 'avenidas de la cuenca' in row[1]: row[34]=1
    if 'invalidez' in row[1]: row[34]=1
    if 'situacion sanitaria' in row[1]: row[34]=1
    if 'tercera edad' in row[1]: row[34]=1
    if 'pension' in row[1]: row[34]=1
    if 'desempleo' in row[1]: row[34]=1
    if 'evacuado' in row[1]: row[34]=1
    if 'minusvalido' in row[1]: row[34]=1
    if 'droga' in row[1]: row[34]=1
    if 'emigra' in row[1]: row[34]=1
    if 'inmigra' in row[1]: row[34]=1
    if 'vivienda' in row[1]: row[34]=1
    if 'matrimoni' in row[1]: row[34]=1
    if 'salud' in row[1]: row[34]=1
    if 'alimentari' in row[1]: row[34]=1
    if 'toxico' in row[1]: row[34]=1
    if 'contaminacion' in row[1]: row[34]=1
    if 'recursos vivos' in row[1]: row[34]=1
    if 'diversidad biologica' in row[1]: row[34]=1
    if 'zonas especialmente protegidas' in row[1]: row[34]=1
    if 'protocolo de montreal' in row[1]: row[34]=1
    if 'medio ambiente' in row[1]: row[34]=1
    if 'derechos humanos' in row[1]: row[34]=1
    if 'protección' in row[1]: row[34]=1
    if 'sanitaria' in row[1]: row[34]=1
    if 'derecho de asilo' in row[1]: row[34]=1


  # Justicia
    if 'justicia' in row[1]: row[35]=1
    if 'escuchas telefonicas' in row[1]: row[35]=1
    if 'defensor del pueblo' in row[1]: row[35]=1
    if 'fiscal' in row[1]: row[35]=1
    if 'judicial' in row[1]: row[35]=1
    if 'juridico' in row[1]: row[35]=1
    if 'jurídico' in row[1]: row[35]=1
    if 'amparo' in row[1]: row[35]=1

  # Internacionales
    if 'internacional' in row[1]: row[36]=1
    if 'europ' in row[1]: row[36]=1
    if 'estados unidos' in row[1]: row[36]=1
    if 'atlantico norte' in row[1]: row[36]=1
    if 'norteamerica' in row[1]: row[36]=1
    if 'la cee' in row[1]: row[36]=1
    if 'jordania' in row[1]: row[36]=1
    if 'naciones unidas' in row[1]: row[36]=1
    if 'interamericana' in row[1]: row[36]=1
    if 'extranjero' in row[1]: row[36]=1
    if 'islandia' in row[1]: row[36]=1
    if 'holand' in row[1]: row[36]=1
    if 'multinacional' in row[1]: row[36]=1
    if 'australia' in row[1]: row[36]=1
    if 'israel' in row[1]: row[36]=1
    if 'artículo k.3' in row[1]: row[36]=1
    if 'artículo k. 3' in row[1]: row[36]=1
    if 'copenhague' in row[1]: row[36]=1
    if 'ocde' in row[1]: row[36]=1
    if 'austria' in row[1]: row[36]=1
    if 'alfonso dastis quecedo' in row[1]: row[36]=1
    if 'convención de la hay' in row[1]: row[36]=1


  # Política Interna
    if 'vasco' in row[1]: row[37]=1
    if 'electoral' in row[1]: row[37]=1
    if 'cataluña' in row[1]: row[37]=1
    if 'andalucia' in row[1]: row[37]=1
    if 'ciudad' in row[1]: row[37]=1
    if 'españa' in row[1]: row[37]=1
    if 'albacete' in row[1]: row[37]=1
    if 'cadiz' in row[1]: row[37]=1
    if 'asturia' in row[1]: row[37]=1
    if 'palmas' in row[1]: row[37]=1
    if 'navarra' in row[1]: row[37]=1
    if 'maspalomas' in row[1]: row[37]=1
    if 'barcelona' in row[1]: row[37]=1
    if 'canaria' in row[1]: row[37]=1
    if 'gibraltar' in row[1]: row[37]=1
    if 'aragon' in row[1]: row[37]=1
    if 'extremadura' in row[1]: row[37]=1
    if 'ibiza' in row[1]: row[37]=1
    if 'castilla' in row[1]: row[37]=1
    if 'autonomia' in row[1]: row[37]=1
    if 'segovia' in row[1]: row[37]=1
    if 'catalana' in row[1]: row[37]=1
    if 'islas baleares' in row[1]: row[37]=1
    if 'illes balears' in row[1]: row[37]=1
    if 'madrid' in row[1]: row[37]=1
    if 'murcia' in row[1]: row[37]=1
    if 'ayuntamiento' in row[1]: row[37]=1
    if 'estatuto de autonomía' in row[1]: row[37]=1
    if 'estatutos de autonomía' in row[1]: row[37]=1
    if 'torremolinos' in row[1]: row[37]=1
    if 'la misión a la luz de su extensión hacia el oeste y el sur del país' in row[1]: row[37]=1
    if 'cuenca' in row[1]: row[37]=1
    if 'autoridades locales' in row[1]: row[37]=1
    if 'comunidades autonomas' in row[1]: row[37]=1
    if 'zoido álvarez' in row[1]: row[37]=1

  # Otros
    if 'hechos acaecidos' in row[1]: row[38]=1
    if 'akáa' in row[1]: row[38]=1
    if 'picasso' in row[1]: row[38]=1
    if 'muerte' in row[1]: row[38]=1
    if 'del señor' in row[1]: row[38]=1
    if '1981' in row[1]: row[38]=1
    if 'preguntas' in row[1]: row[38]=1
    if 'convivencia' in row[1]: row[38]=1
    if 'simon bolivar' in row[1]: row[38]=1
    if 'interpelaciones urgentes' in row[1]: row[38]=1
    if 'recomendaciones' in row[1]: row[38]=1
    if 'screto profesional' in row[1]: row[38]=1
    if 'octubre de 1986' in row[1]: row[38]=1
    if 'sucesivos.' in row[1]: row[38]=1
    if 'debate sobre el estado de la nacion' in row[1]: row[38]=1
    if '30 de noviembre de 1999' in row[1]: row[38]=1
    if 'septiembre de 2000' in row[1]: row[38]=1
    if 'propuestas de resolución' in row[1]: row[38]=1
    if 'nan' in row[1]: row[38]=1
    if 'recuperacion automatica' in row[1]: row[38]=1
    if 'del grupo socialista del' in row[1]: row[38]=1
    if 'del grupo pa' in row[1]: row[38]=1
    if 'coag y upa' in row[1]: row[38]=1
    if 'rumores' in row[1]: row[38]=1
    if 'psoe que provoca' in row[1]: row[38]=1
    if 'viii legislatura' in row[1]: row[38]=1
    if 'públicos y privados' in row[1]: row[38]=1
    if 'modificación' in row[1]: row[38]=1
    if 'públicas' in row[1]: row[38]=1
    if 'acudir estos' in row[1]: row[38]=1
    if 'públicas' in row[1]: row[38]=1
    if 'swd (2013) 472 final' in row[1]: row[38]=1
    if 'zurita expósito' in row[1]: row[38]=1
    if 'condolencia' in row[1]: row[38]=1
    if 'convención de la haya de 1954' in row[1]: row[38]=1
    if 'xviii, yo creo que hay que irse a los siglos xix o xx' in row[1]: row[38]=1
    if 'dé mayo de .—núm.' in row[1]: row[38]=1
    if '| de octubre de .—núm.' in row[1]: row[38]=1
    if 'dé mayo de .—núm. ' in row[1]: row[38]=1
    if 'final] [swd ()  final] [swd ()  final].' in row[1]: row[38]=1

    lista.append(row)

base = pd.DataFrame(lista, columns=base.columns)

In [ ]:
base['topic_designado_a_mano']=0
lista=[]
for row in base.values:
  if row[24]==1: row[39]=1
  if row[25]==1: row[39]=2
  if row[26]==1: row[39]=3
  if row[27]==1: row[39]=4
  if row[28]==1: row[39]=5
  if row[29]==1: row[39]=6
  if row[30]==1: row[39]=7
  if row[31]==1: row[39]=8
  if row[32]==1: row[39]=9
  if row[33]==1: row[39]=10
  if row[34]==1: row[39]=11
  if row[35]==1: row[39]=12
  if row[36]==1: row[39]=13
  if row[37]==1: row[39]=14
  if row[38]==1: row[39]=15
  lista.append(row)

base = pd.DataFrame(lista, columns=base.columns)

In [ ]:
# Por LEGISLATURA por DESIGNACION MANUAL
legislaturas=['I', 'II', 'III', 'IV', 'V', 'VI', 'VII','VIII', 'IX', 'X', 'XI', 'XII','XIII']
topics_por_legislatura=pd.DataFrame()
for x in legislaturas:
  base_legislatura=base[base.legislatura==x]
  topics_por_legislatura[x]=base_legislatura['topic_designado_a_mano'].value_counts().sort_index()
topics_por_legislatura.to_csv('topics_por_legislatura_por_designacionamano.csv')

In [ ]:
# Por AÑO por DESIGNACIÓN MANUAL
topics_por_año=pd.DataFrame()
for x in range (1979, 2020):
  base_year=base[pd.DatetimeIndex(base.fecha).year==x]
  topics_por_año[x]=base_year['topic_designado_a_mano'].value_counts().sort_index()
topics_por_año.to_csv('topics_por_año_por_designacionamano.csv')

# Bajada

In [21]:
from google.colab import files
files.download('/content/topics_por_año_por_weights_por_ordendeldia_snowball.csv')
files.download('/content/topics_por_legislatura_por_weights_por_ordendeldia_snowball.csv')
files.download('/content/topics_por_año_por_frex_por_ordendeldia_snowball.csv')
files.download('/content/topics_por_legislatura_por_frex_por_ordendeldia_snowball.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>